
# 基于深度神经网络的流固耦合系统建模方法

## 环境安装

本案例要求 **MindSpore >= 2.0.0** 版本以调用如下接口: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*。具体请查看[MindSpore安装](https://www.mindspore.cn/install)。

此外，你需要安装 **MindFlow >=0.1.0** 版本。如果当前环境还没有安装，请按照下列方式选择后端和版本进行安装。

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 概述

飞行器气动弹性问题是研究飞行器结构与气动力相互耦合关系，是一种典型的流固耦合（fluid-structure interaction, FSI）问题。高精确度计算流体力学技术（CFD）能够准确地模拟结构周围流场演化过程从而获得结构受力情况,但是数量巨大的网格导致计算成本极高。许多研究者尝试使用数据驱动的方法构建流场演化模型，实现较高精度流场的快速预测，从而提高流固耦合系统模拟效率。近年来迅速发展的深度神经网络技术依赖于其强大的非线性学习能力以及深度特征捕捉能力，在流场建模问题中已经取得了诸多成功应用。其中流场重构通过构建几何形状和流动工况至空间点处的流场信息之间的映射模型，实现不同流场的快速预测，因其能快速给出当前流场状态而备受关注。

为了高效解决流固耦合问题的流场重构，本文将神经网络模型与计算结构动力学方程耦合，实现了流固耦合系统的建模，进一步改进神经网络结构，优化数据结构，从而获得更高精度的流场预测结果，实现更准确的流固耦合响应预测。

## 问题描述

传统的流固耦合数值仿真框架由计算流体力学求解器和计算固体力学求解器两部分组成，两个求解器分别在流体域和固体域求解下一时刻流体和结构的状态，并在交界面进行信息传递作为下一步计算的输入，耦合过程如下图所示。本文提出的基于深度神经网络的流固耦合建模框架仍然采用相同的策略，该框架使用深度神经网络代替CFD求解器来预测流场演化，结构响应仍由CSD求解器计算得到，结构位移和流场表面压力在深度神经网络和计算固体力学求解器之间传递。

## 技术路径

MindFlow求解该问题的具体流程如下：

1.根据CFD数值模拟结果创建数据集。

2.使用MindSpore深度学习框架构建模型。

3.定义优化器与损失函数。

4.使用MindSpore的即时编译等加速模型训练。

5.利用训练好的模型进行推理和可视化。

![p1.png](./images/p1.png)

## 模型架构

HDNN的基本框架由卷积神经网络（CNN）、卷积长短期记忆网络（ConvLSTM）和反卷积神经网络（DeCNN）组成。CNN降低了时间序列流场的维数，实现特征提取；ConvLSTM学习低维时空特征并进行预测；最后，DeCNN实现预测流场的重建

+ 输入层：当前流场状态和边界条件；
+ 卷积层：捕获流场的空间特征并降低维数，使用低维流场特征预测流场演化可以提高计算效率；
+ LSTM层：根据捕获的当前时刻流场特征和结构运动条件预测下一时刻的流场特征；
+ 反卷积输出层：将预测流场的低维特征恢复到高维空间，通过多层DeCNN重构下一时刻的瞬态流场，并输出可视化预测结果

![HDNN.jpg](./images/HDNN.jpg)

## 训练数据集

数据集由非定常二维圆柱绕流的数值仿真流场数据构建的多维矩阵流场快照矩阵构建而成

+ 流场中的运动结构（圆柱）在竖直方向做一维简谐运动。对二维圆柱流场物理建模、网格离散/划分并采用雷诺时均模拟方法求解控制方程获取流场信息。将流场物理量无量纲化，并在采样区域中放置网格采样点，获得用于训练和测试的样本集
+ 每张流场快照包含3个通道，代表流场的压强分布信息、水平速度信息、竖直速度信息
+ 数据集：[下载位置](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/fluid_structure_interaction/)

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, data_sink, jit
from mindflow.utils import load_yaml_config

from src import generate_dataset, AEnet, save_loss_curve

## 训练环境

+ 训练采用Mindspore框架的静态图模式（GRAPH）
+ 在CPU、GPU或Ascend进行训练

In [2]:
set_seed(0)
np.random.seed(0)

## 训练超参数

从config中获得模型、数据、优化器的超参

In [3]:
parser = argparse.ArgumentParser(description="cylinder around flow ROM")

parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./summary")
parser.add_argument("--device_target", type=str, default="Ascend", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'GPU','Ascend'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--data_list", type=list, default=['5.0', '5.5', '6.0', '6.5'], help="The type for training")
parser.add_argument('--batch_size', type=int, default=32, help="batch size")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")

args = parser.parse_args()

context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs, save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target, device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

config = load_yaml_config(args.config_file_path)
data_params = config["data"]
model_params = config["model"]
optimizer_params = config["optimizer"]

## 训练过程文件保存路径

将训练好的模型文件每隔一定训练次数保存在文件夹下

In [4]:
ckpt_dir = optimizer_params["ckpt_dir"]
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## 构建神经网络及优化器

神经网络的卷积层共有12层，ConvLSTM有1层，反卷积共有12层

损失函数使用均方误差（Mean Squared Error）损失函数，优化器使用Adam（Adaptive Moment Estimation）优化算法

In [5]:
model = AEnet(in_channels=model_params["in_channels"],
              num_layers=model_params["num_layers"],
              kernel_size=model_params["kernel_size"],
              num_convlstm_layers=model_params["num_convlstm_layers"])

loss_func = nn.MSELoss()
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=optimizer_params["lr"])
if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, 'O1')
else:
    loss_scaler = None

## 训练框架

定义前向传播函数forward_fn，将预测值和真值比较得到损失值loss并返回

In [6]:
def forward_fn(inputs, velocity, ur, label):
    pred = model(inputs, velocity, ur)
    loss = loss_func(pred, label)

    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)

## 数据集加载

给generate_dataset传参，得到训练数据集和验证数据集

In [7]:
print(f"==================Load data sample ===================")
dataset_train, dataset_eval = generate_dataset(data_params["data_dir"],
                                               data_params["time_steps"],
                                               args.data_list)
print(f"======================End Load========================\n")

## 数据下沉及模型训练

定义train_step和eval_step并使用data_sink加速训练，输出训练过程的损失值和使用时间，并每隔一定训练轮次保存模型文件

In [8]:
print(f"====================Start train=======================")
@jit
def train_step(inputs, velocity, ur, label):
    loss, grads = grad_fn(inputs, velocity, ur, label)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, optimizer(grads))
    return loss

@jit
def eval_step(inputs, velocity, ur, label):
    loss = forward_fn(inputs, velocity, ur, label)
    loss = ops.sqrt(loss)
    return loss

train_sink_process = data_sink(train_step, dataset_train, sink_size=1)
eval_sink_process = data_sink(eval_step, dataset_eval, sink_size=1)
train_data_size, eval_data_size = dataset_train.get_dataset_size(), dataset_eval.get_dataset_size()

avg_train_losses = []
avg_valid_losses = []

for epoch in range(1, optimizer_params["epochs"] + 1):
    train_losses = 0
    valid_losses = 0

    local_time_beg = time.time()
    model.set_train(True)

    for _ in range(train_data_size):
        step_train_loss = ops.squeeze(train_sink_process(), axis=())
        step_train_loss = step_train_loss.asnumpy().item()
        train_losses += step_train_loss

    train_loss = train_losses / train_data_size
    avg_train_losses.append(train_loss)

    print(f"epoch: {epoch}, epoch average train loss: {train_loss :.6f}, "
          f"epoch time: {(time.time() - local_time_beg):.2f}s")

    if epoch % optimizer_params["eval_interval"] == 0:
        print(f"=================Start Evaluation=====================")

        eval_time_beg = time.time()
        model.set_train(False)
        for _ in range(eval_data_size):
            step_eval_loss = ops.squeeze(eval_sink_process(), axis=())
            step_eval_loss = step_eval_loss.asnumpy().item()
            valid_losses += step_eval_loss

        valid_loss = valid_losses / eval_data_size
        avg_valid_losses.append(valid_loss)

        print(f"epoch: {epoch}, epoch average valid loss: {valid_loss :.6f}, "
              f"epoch time: {(time.time() - eval_time_beg):.2f}s")
        print(f"==================End Evaluation======================")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/net_{epoch}.ckpt")

save_loss_curve(avg_train_losses, 'Epoch', 'avg_train_losses', 'Avg_train_losses Curve', 'Avg_train_losses.png')
save_loss_curve(avg_valid_losses, 'Epoch', 'avg_valid_losses', 'Avg_valid_losses Curve', 'Avg_valid_losses.png')

print(f"=====================End train========================")

## 设置训练条件 传参

当运行该文件时，通过参数解析器传入必要参数，开始训练，并打印进程和设备id，以及训练总时间

In [9]:
if __name__ == "__main__":
    print("Process ID:", os.getpid())
    print(f"device id: {args.device_id}")
    start_time = time.time()
    train()
    print(f"End-to-End total time: {(time.time() - start_time):.2f}s")

## 预测流场结果可视化

+ 动边界流场预测通过执行eval.py开始预测，耦合模型可在仅给定初始流场状态和圆柱位置情况下，完成整个流固耦合演化过程的预测任务
+ 下图为训练完备的HDNN模型实现对一个周期内不同时刻深度神经网络的流场预测状态

![pred_cycle_puv.jpg](./images/pred_cycle_puv.jpg)